In [2]:
import pandas as pd
import numpy as np
import ta
from ta import add_all_ta_features
from pandas_datareader import data as pdr
from sklearn import preprocessing
from keras.models import Model, Sequential
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense, Activation
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
from keras import regularizers, optimizers

In [25]:
# Load the whole data set
data = pd.read_csv('./datasets/data.csv')
list_of_stocks = ['JNJ', 'KO', 'PG', 'TRV', 'MRK', 'IBM', 'VZ', 'WMT', 'MMM', 'XOM',
'DIS', 'MCD', 'MSFT', 'CSCO', 'PFE', 'V', 'HD', 'CAT', 'NKE', 'JPM']

In [26]:
data = data[data['tic'].isin(list_of_stocks)]

In [27]:
data.tic.unique()

array(['MSFT', 'JPM', 'V', 'PG', 'NKE', 'DIS', 'HD', 'WMT', 'IBM', 'MRK',
       'KO', 'CAT', 'TRV', 'JNJ', 'MCD', 'VZ', 'CSCO', 'XOM', 'MMM',
       'PFE'], dtype=object)

In [28]:
# Define a Function for adding technical indicators

def add_features(data, feature_list, short_names):
    """
    Function to add technical indicators for features
    -Takes in a dataset with Open, High, Low, Close and Volume
    -Also takes in a list of the technical indicators to be added 
     as well as a list of the shortened indicator names
    """
    
    # list of column names to filter the features
    data_col_names = list(data.columns)
    filter_names = data_col_names + feature_list
    col_rename = data_col_names +  short_names
    
    # Add technical indicators using the ta Library
    data = add_all_ta_features(data, open="open", high="high", 
    low="low", close="close", volume="volume") 
    
    # Filter the Indicators with the required features
    data = data[filter_names]
    data.columns = col_rename # rename the columns to use shortened indicator names
    data = data.dropna()
    
    return data

In [29]:
# List of Features to add
feature_list= ['volatility_atr','volatility_bbw','volume_obv','volume_cmf',
               'trend_macd', 'trend_adx', 'trend_sma_fast', 
               'trend_ema_fast', 'trend_cci', 'momentum_rsi']

# Short names of the features
short_names = ['atr', 'bbw','obv','cmf','macd', 'adx', 'sma', 'ema', 'cci', 'rsi']

#feature_list= ['volatility_atr','volatility_bbw','volume_obv','volume_cmf','trend_macd']

# Short names of the features
#short_names = ['atr', 'bbw','obv','cmf','macd']

In [30]:
# Add Indicators to our dataset
data_with_features = data.copy()

data_with_features = add_features(data_with_features, feature_list, short_names)
data_with_features.head()

c:\Users\Usuario\Documents\deep_learning_asset_portfolio_management\env\lib\site-packages\ta\trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
c:\Users\Usuario\Documents\deep_learning_asset_portfolio_management\env\lib\site-packages\ta\trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


,date,open,high,low,close,volume,tic,atr,bbw,obv,cmf,macd,adx,sma,ema,cci,rsi
3046,2009-02-06,19.160000,19.930000,19.059999,14.961291,86746000.0,MSFT,4.796015,19.908847,666119400.0,-11.968562,-0.134506,0.000000,13.697392,14.104219,83.765563,57.740745
3047,2009-02-09,19.639999,19.770000,19.260000,14.793870,52196400.0,MSFT,4.797284,19.824320,613923000.0,-12.195378,-0.069528,0.000000,13.845153,14.210319,82.279873,55.710462
3048,2009-02-10,19.250000,19.799999,18.700001,14.306821,83953200.0,MSFT,4.818169,19.364400,529969800.0,-11.818747,-0.056679,19.813499,13.946619,14.225165,54.030264,50.182408
3049,2009-02-11,18.940001,19.490000,18.920000,14.618835,58599000.0,MSFT,4.854670,18.618891,588568800.0,-11.652975,-0.021077,18.755084,14.046818,14.285730,65.615975,53.374297
3050,2009-02-12,18.969999,19.320000,18.540001,14.656886,75323200.0,MSFT,4.839319,18.760384,663892000.0,-11.492100,0.010092,18.256305,14.148285,14.342831,50.931498,53.763363


In [31]:
feature_list = list(data_with_features.columns)[7:]
feature_list

['atr', 'bbw', 'obv', 'cmf', 'macd', 'adx', 'sma', 'ema', 'cci', 'rsi']

In [32]:
def add_cov_matrix(df):
    """
    Function to add Coveriance Matrices as part of the defined states
    """
    # Sort the data and index by date and tic
    df=df.sort_values(['date','tic'],ignore_index=True) 
    df.index = df.date.factorize()[0]
    
    cov_list = [] # create empty list for storing coveriance matrices at each time step
    
    # look back for constructing the coveriance matrix is one year
    lookback=252
    for i in range(lookback,len(df.index.unique())):
        data_lookback = df.loc[i-lookback:i,:]
        price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
        return_lookback = price_lookback.pct_change().dropna()
        covs = return_lookback.cov().values 
        covs = covs#/covs.max()
        cov_list.append(covs)
        
    df_cov = pd.DataFrame({'date':df.date.unique()[lookback:],'cov_list':cov_list})
    df = df.merge(df_cov, on='date')
    df = df.sort_values(['date','tic']).reset_index(drop=True)
    
    return df

In [33]:
# Add Covariance Matrices to our dataset
data_with_features_covs = data_with_features.copy()
data_with_features_covs = add_cov_matrix(data_with_features_covs)
df = data_with_features_covs

In [34]:
# Get the list of all the features
features_list = list(df.columns)[7:-1]
print(features_list)
# construct a data frame of features
features_df = df[features_list]
features_df.index = df['date']
features_df.head()

['atr', 'bbw', 'obv', 'cmf', 'macd', 'adx', 'sma', 'ema', 'cci', 'rsi']


,atr,bbw,obv,cmf,macd,adx,sma,ema,cci,rsi
date,,,,,,,,,,
2009-12-31,16.411522,4.599234,1.751191e+10,-35.806296,0.044336,10.020770,41.918410,41.881607,-50.733730,44.302408
2009-12-31,6.172343,4.656155,2.310719e+10,-36.494340,0.060804,8.984294,17.752903,17.857885,57.386845,53.264455
2009-12-31,4.433756,10.239793,1.007232e+10,-17.293213,0.566996,28.281289,28.071835,27.909763,77.608718,62.419287
2009-12-31,6.844032,7.539178,1.128392e+10,-30.420664,0.374300,19.200721,22.384340,22.300831,58.112405,58.501256
2009-12-31,40.081142,5.281060,1.315387e+10,-68.591339,0.972473,14.028633,90.417915,90.721556,118.276233,59.004777


In [35]:
print(features_df.shape)
print(df.shape)

(55380, 10)
(55380, 18)


In [36]:
features_array = np.array(features_df)
features_scaler = preprocessing.MinMaxScaler()
features_normalised = features_scaler.fit_transform(features_array)
features_normalised.shape

(55380, 10)

In [37]:
# rescale the features array
features_normalised = features_normalised.reshape(-1,20,10)

In [38]:
# define model
model = Sequential()
model.add(LSTM(4, activation='relu', input_shape=(20,10)))
model.add(RepeatVector(20))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(10)))
model.compile(optimizer='adam', loss='mse')

In [39]:
# fit model
model.fit(features_normalised, features_normalised, epochs=100, verbose=1)
plot_model(model, show_shapes=True, to_file='./results/reconstruct_lstm_autoencoder.png')

Epoch 1/100
87/87 [==============================] - 3s 15ms/step - loss: 0.0595
Epoch 2/100
87/87 [==============================] - 1s 14ms/step - loss: 0.0230
Epoch 3/100
87/87 [==============================] - 1s 13ms/step - loss: 0.0188
Epoch 4/100
87/87 [==============================] - 1s 13ms/step - loss: 0.0175
Epoch 5/100
87/87 [==============================] - 1s 13ms/step - loss: 0.0167
Epoch 6/100
87/87 [==============================] - 1s 14ms/step - loss: 0.0159
Epoch 7/100
87/87 [==============================] - 1s 14ms/step - loss: 0.0146
Epoch 8/100
87/87 [==============================] - 1s 13ms/step - loss: 0.0125
Epoch 9/100
87/87 [==============================] - 1s 14ms/step - loss: 0.0114
Epoch 10/100
87/87 [==============================] - 1s 13ms/step - loss: 0.0107
Epoch 11/100
87/87 [==============================] - 1s 13ms/step - loss: 0.0101
Epoch 12/100
87/87 [==============================] - 1s 13ms/step - loss: 0.0097
Epoch 13/100
87/87 [=====

In [ ]:
model.summary()

In [40]:
# connect the encoder LSTM as the output layer
model_feature = Model(inputs=model.inputs, outputs=model.layers[1].output)
plot_model(model_feature, show_shapes=True, show_layer_names=True, to_file='./results/lstm_encoder.png')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [41]:
# get the feature vector for the input sequence
yhat = model_feature.predict(features_normalised)
print(yhat.shape)

87/87 [==============================] - 0s 3ms/step
(2769, 20, 4)


In [42]:
# reshape the vector
features_reduced = yhat.reshape(-1,4)

In [43]:
# Copy original data frame and drop the original features
df_reduced = df.copy()
df_reduced = df_reduced.drop(features_list, axis=1)
df_reduced.head()

,date,open,high,low,close,volume,tic,cov_list
0,2009-12-31,57.599998,57.959999,56.990002,41.246521,3859700.0,CAT,"[[0.0010639027833897088, 0.0004996216911050764..."
1,2009-12-31,24.100000,24.170000,23.940001,17.903095,25208100.0,CSCO,"[[0.0010639027833897088, 0.0004996216911050764..."
2,2009-12-31,32.270000,32.750000,32.220001,28.090706,19651700.0,DIS,"[[0.0010639027833897088, 0.0004996216911050764..."
3,2009-12-31,29.090000,29.370001,28.889999,22.307234,7437100.0,HD,"[[0.0010639027833897088, 0.0004996216911050764..."
4,2009-12-31,132.410004,132.850006,130.750000,91.085335,4223400.0,IBM,"[[0.0010639027833897088, 0.0004996216911050764..."


In [44]:
# convert the reduced features to a data frame and merge with the original data frame
features_reduced_df = pd.DataFrame(features_reduced, columns=['f01','f02','f03','f04'])

In [45]:
df_reduced[['f01','f02','f03','f04']] = features_reduced_df[['f01','f02','f03','f04']]
df_reduced.head()
data_df = df_reduced.copy()

In [46]:
data_df

,date,open,high,low,close,volume,tic,cov_list,f01,f02,f03,f04
0,2009-12-31,57.599998,57.959999,56.990002,41.246521,3859700.0,CAT,"[[0.0010639027833897088, 0.0004996216911050764...",3.706141,1.601194,0.707921,2.725062
1,2009-12-31,24.100000,24.170000,23.940001,17.903095,25208100.0,CSCO,"[[0.0010639027833897088, 0.0004996216911050764...",3.706141,1.601194,0.707921,2.725062
2,2009-12-31,32.270000,32.750000,32.220001,28.090706,19651700.0,DIS,"[[0.0010639027833897088, 0.0004996216911050764...",3.706141,1.601194,0.707921,2.725062
3,2009-12-31,29.090000,29.370001,28.889999,22.307234,7437100.0,HD,"[[0.0010639027833897088, 0.0004996216911050764...",3.706141,1.601194,0.707921,2.725062
4,2009-12-31,132.410004,132.850006,130.750000,91.085335,4223400.0,IBM,"[[0.0010639027833897088, 0.0004996216911050764...",3.706141,1.601194,0.707921,2.725062
...,...,...,...,...,...,...,...,...,...,...,...,...
55375,2020-12-30,138.949997,139.899994,138.850006,139.080002,1253800.0,TRV,"[[0.0008016224880885106, 0.0005013084093293419...",7.507486,1.586209,0.935624,2.299339
55376,2020-12-30,216.000000,220.389999,215.649994,218.360001,8875100.0,V,"[[0.0008016224880885106, 0.0005013084093293419...",7.507486,1.586209,0.935624,2.299339
55377,2020-12-30,58.830002,58.939999,58.060001,58.139999,18259800.0,VZ,"[[0.0008016224880885106, 0.0005013084093293419...",7.507486,1.586209,0.935624,2.299339
55378,2020-12-30,144.880005,145.149994,143.940002,144.179993,6250400.0,WMT,"[[0.0008016224880885106, 0.0005013084093293419...",7.507486,1.586209,0.935624,2.299339


In [47]:
# Define the start and end dates for the train and test data

train_pct = 0.8 # percentage of train data
date_list = list(data_df.date.unique()) # List of dates in the data

date_list_len = len(date_list) # len of the date list
train_data_len = int(train_pct * date_list_len) # length of the train data

train_start_date = date_list[0]
train_end_date = date_list[train_data_len]

test_start_date = date_list[train_data_len+1]
test_end_date = date_list[-1]


In [48]:
print('Training Data: ', 'from ', train_start_date, ' to ', train_end_date)
print('Testing Data: ', 'from ', test_start_date, ' to ', test_end_date)

Training Data:  from  2009-12-31  to  2018-10-18
Testing Data:  from  2018-10-19  to  2020-12-30


In [49]:
from utils import data_split

train_data = data_split(data_df, train_start_date, train_end_date)
test_data = data_split(data_df, test_start_date, test_end_date)
train_df = train_data.copy()
test_df = test_data.copy()

In [50]:
import env_portfolio
from env_portfolio import StockPortfolioEnv

import models
from models import DRLAgent

In [51]:
stock_dimension = len(train_df.tic.unique())
state_space = stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")
weights_initial = [1/stock_dimension]*stock_dimension
tech_indicator_list = ['f01','f02','f03','f04']

Stock Dimension: 20, State Space: 20


In [52]:
env_kwargs = {
    "hmax": 500, 
    "initial_amount": 1000000, 
    "transaction_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": tech_indicator_list, 
    "action_space": stock_dimension, 
    "reward_scaling": 0,
    'initial_weights': [1/stock_dimension]*stock_dimension
}

In [53]:
e_train_gym = StockPortfolioEnv(df = train_df, **env_kwargs)
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


c:\Users\Usuario\Documents\deep_learning_asset_portfolio_management\env\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [54]:
# initialize
agent = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 5, "ent_coef": 0.005, "learning_rate": 0.0002}
model_a2c = agent.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 5, 'ent_coef': 0.005, 'learning_rate': 0.0002}
Using cpu device


In [55]:
trained_a2c = agent.train_model(model=model_a2c, 
                                tb_log_name='a2c',
                                total_timesteps=50000)

Logging to tensorboard_log/a2c\a2c_2
------------------------------------
| time/                 |          |
|    fps                | 378      |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -28.3    |
|    explained_variance | 0        |
|    learning_rate      | 0.0002   |
|    n_updates          | 99       |
|    policy_loss        | 1.04e+08 |
|    std                | 0.997    |
|    value_loss         | 1.59e+13 |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 379      |
|    iterations         | 200      |
|    time_elapsed       | 2        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -28.3    |
|    explained_variance | 0        |
|    learning_rate      | 0.0002   |
|    n_updates          | 199      |
|

In [56]:
# A2C Train Model
e_trade_gym = StockPortfolioEnv(df = train_df, **env_kwargs)
env_trade, obs_trade = e_trade_gym.get_sb_env()

a2c_train_daily_return, a2c_train_weights = DRLAgent.DRL_prediction(model=trained_a2c,
                        test_data = train_df,
                        test_env = env_trade,
                        test_obs = obs_trade)

begin_total_asset:1000000
end_total_asset:3486836.2855546037
Sharpe:  1.140399454061648


In [57]:
# A2C Test Model
e_trade_gym = StockPortfolioEnv(df = test_df, **env_kwargs)
env_trade, obs_trade = e_trade_gym.get_sb_env()

a2c_test_daily_return, a2c_test_weights = DRLAgent.DRL_prediction(model=trained_a2c,
                        test_data = test_df,
                        test_env = env_trade,
                        test_obs = obs_trade)

c:\Users\Usuario\Documents\deep_learning_asset_portfolio_management\env\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


begin_total_asset:1000000
end_total_asset:1312209.4334361842
Sharpe:  0.6337230352867573


In [58]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.0001,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.0001, 'batch_size': 128}
Using cpu device


In [ ]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

In [ ]:
# PPO Train Model
e_trade_gym = StockPortfolioEnv(df = train_df, **env_kwargs)
env_trade, obs_trade = e_trade_gym.get_sb_env()

ppo_train_daily_return, ppo_train_weights = DRLAgent.DRL_prediction(model=trained_ppo,
                        test_data = train_df,
                        test_env = env_trade,
                        test_obs = obs_trade)

In [ ]:
# PPO Test Model
e_trade_gym = StockPortfolioEnv(df = test_df, **env_kwargs)
env_trade, obs_trade = e_trade_gym.get_sb_env()

ppo_test_daily_return, ppo_test_weights = DRLAgent.DRL_prediction(model=trained_ppo,
                        test_data = test_df,
                        test_env = env_trade,
                        test_obs = obs_trade)

In [ ]:
############################### asi hay que operar en produccion #########################
env_kwargs_test = {
    "hmax": 500, 
    "initial_amount": 5000, 
    "transaction_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": tech_indicator_list, 
    "action_space": stock_dimension, 
    "reward_scaling": 0,
    'initial_weights': [1/stock_dimension]*stock_dimension
}

e_trade_gym = StockPortfolioEnv(df = test_df, **env_kwargs_test)
env_trade, obs_trade = e_trade_gym.get_sb_env()

ppo_test_daily_return, ppo_test_weights = DRLAgent.DRL_prediction(model=trained_ppo,
                        test_data = test_df,
                        test_env = env_trade,
                        test_obs = obs_trade)

In [64]:
agent = DRLAgent(env = env_train)
DDPG_PARAMS = {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}
model_ddpg = agent.get_model("ddpg",model_kwargs = DDPG_PARAMS)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device


In [66]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000)

Logging to tensorboard_log/ddpg\ddpg_1
begin_total_asset:1000000
end_total_asset:3422869.824724422
Sharpe:  1.1274960483557201
begin_total_asset:1000000
end_total_asset:3439595.4824811267
Sharpe:  1.1009502911626945
begin_total_asset:1000000
end_total_asset:3439595.4824811267
Sharpe:  1.1009502911626945
begin_total_asset:1000000
end_total_asset:3439595.4824811267
Sharpe:  1.1009502911626945
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 44        |
|    time_elapsed    | 200       |
|    total_timesteps | 8860      |
| train/             |           |
|    actor_loss      | -4.97e+07 |
|    critic_loss     | 1.86e+12  |
|    learning_rate   | 0.001     |
|    n_updates       | 6645      |
----------------------------------
begin_total_asset:1000000
end_total_asset:3439595.4824811267
Sharpe:  1.1009502911626945
begin_total_asset:1000000
end_total_asset:3439595.4824811267
Sharpe:  1.1009502911626945
begin_to

In [67]:
# DDPG Test Model
e_trade_gym = StockPortfolioEnv(df = test_df, **env_kwargs)
env_trade, obs_trade = e_trade_gym.get_sb_env()

ddpg_test_daily_return, ddpg_test_weights = DRLAgent.DRL_prediction(model=trained_ddpg,
                        test_data = test_df,
                        test_env = env_trade,
                        test_obs = obs_trade)

c:\Users\Usuario\Documents\deep_learning_asset_portfolio_management\env\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


begin_total_asset:1000000
end_total_asset:1315830.847142868
Sharpe:  0.6394863792036135


In [69]:
a2c_test_portfolio = a2c_test_daily_return.copy()
a2c_test_returns = a2c_test_daily_return.copy()

ppo_test_portfolio = ppo_test_daily_return.copy()
ppo_test_returns = ppo_test_daily_return.copy()

ddpg_test_portfolio = ddpg_test_daily_return.copy()
ddpg_test_returns = ddpg_test_daily_return.copy()

In [75]:
a2c_test_cum_returns = (1 + a2c_test_returns['daily_return']).cumprod()
a2c_test_cum_returns.name = 'Portfolio 1: a2c Model'

In [76]:
ppo_test_cum_returns = (1 + ppo_test_returns['daily_return']).cumprod()
ppo_test_cum_returns.name = 'Portfolio 2: ppo Model'

In [77]:
ddpg_test_cum_returns = (1 + ddpg_test_returns['daily_return']).cumprod()
ddpg_test_cum_returns.name = 'Portfolio 3: ddpg Model'

In [73]:
import matplotlib.pylab as plt

In [78]:
# Plot the culmulative returns of the portfolios
fig, ax = plt.subplots(figsize=(10,4))

a2c_test_cum_returns.plot(ax=ax, color='darkorange', alpha=.4)
ppo_test_cum_returns.plot(ax=ax, color='green', alpha=.4)
ddpg_test_cum_returns.plot(ax=ax, color='purple', alpha=.4)
plt.legend(loc="best")
plt.grid(True)
ax.set_ylabel("cummulative return")
ax.set_title("Backtest based on the data from 2018-10-19 to 2020-12-30", fontsize=14)
fig.savefig('results/back_test_on_test_data.png')

In [81]:
# from backtest import BackTestStats, BaselineStats, BackTestPlot, backtest_strat, baseline_strat
# from backtest import backtest_strat
from pyfolio import timeseries

In [82]:
def backtest_strat(df):
    strategy_ret = df.copy()
    strategy_ret["date"] = pd.to_datetime(strategy_ret["date"])
    strategy_ret.set_index("date", drop=False, inplace=True)
    strategy_ret.index = strategy_ret.index.tz_localize("UTC")
    del strategy_ret["date"]
    ts = pd.Series(strategy_ret["daily_return"].values, index=strategy_ret.index)
    return ts

In [83]:
def portfolio_stats(portfolio_returns):
    # Pass the returns into a dataframe
    port_rets_df = pd.DataFrame(portfolio_returns)
    port_rets_df = port_rets_df.reset_index()
    port_rets_df.columns = ['date','daily_return']
    
    #Use the FinRL Library to get the Portfolio Returns
    #This makes use of the Pyfolio Library
    
    DRL_strat = backtest_strat(port_rets_df)
    perf_func = timeseries.perf_stats 
    perf_stats_all = perf_func( returns=DRL_strat, 
                                  factor_returns=DRL_strat, 
                                    positions=None, transactions=None, turnover_denom="AGB")
    perf_stats_all = pd.DataFrame( perf_stats_all)
    perf_stats_all.columns = ['Statistic']
    return perf_stats_all

In [84]:
# Get the Portfolio Statistics for all the portfolios
portfolios_returns_dict = {
                          'a2c Model': a2c_test_returns['daily_return'],
                          'ppo Model': ppo_test_returns['daily_return'],
                          'ddpg Model': ddpg_test_returns['daily_return']}

portfolios_stats = pd.DataFrame()
for i,j in portfolios_returns_dict.items():
    port_stats = portfolio_stats(j)
    portfolios_stats[i] = port_stats['Statistic']

In [85]:
portfolios_stats

,a2c Model,ppo Model,ddpg Model
Annual return,0.132064,0.141418,0.133489
Cumulative returns,0.312209,0.336077,0.315831
Annual volatility,0.242217,0.237239,0.241803
Sharpe ratio,0.633723,0.676522,0.639486
Calmar ratio,0.411601,0.464482,0.440018
Stability,0.396161,0.499613,0.495449
Max drawdown,-0.320855,-0.304465,-0.303372
Omega ratio,1.143112,1.152735,1.143115
Sortino ratio,0.892366,0.958384,0.905397
Skew,-0.367429,-0.259498,-0.290084


In [86]:
ppo_test_returns = ppo_test_returns.set_index('date')
ddpg_test_returns = ddpg_test_returns.set_index('date')

In [87]:
# Getting the best performing portfolio

ps_cum = [a2c_test_cum_returns, ppo_test_cum_returns,ddpg_test_cum_returns]
ps = [a2c_test_returns['daily_return'], ppo_test_returns['daily_return'], ddpg_test_returns['daily_return']]

final_return = []
for p in ps_cum:
    final_return.append(p.iloc[-1])
    
id_ = np.argmax(final_return)
best_p = ps[id_]
best_p.name = (ps_cum[id_]).name 

print("Best portfolio: ",  best_p.name)
print("Final cumulative return: {:.2f} ".format(final_return[id_]))

Best portfolio:  Portfolio 2: ppo Model
Final cumulative return: 1.34 


In [ ]:
env_kwargs_test = {
    "hmax": 500, 
    "initial_amount": 5000, 
    "transaction_cost_pct": 0.001, 
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": tech_indicator_list, 
    "action_space": stock_dimension, 
    "reward_scaling": 0,
    'initial_weights': [1/stock_dimension]*stock_dimension
}

e_trade_gym = StockPortfolioEnv(df = test_df, **env_kwargs_test)
env_trade, obs_trade = e_trade_gym.get_sb_env()

ppo_test_daily_return, ppo_test_weights = DRLAgent.DRL_prediction(model=trained_ppo,
                        test_data = test_df,
                        test_env = env_trade,
                        test_obs = obs_trade)

In [89]:
type(trained_ppo)

stable_baselines3.ppo.ppo.PPO

In [94]:
import pickle
pickle.dump(test_df, open('rl_models_test_data.pkl', 'wb'))

In [96]:
trained_a2c.save("trained_a2c_model.zip")

In [91]:
trained_ppo.save("trained_ppo_model.zip")

In [97]:
trained_ddpg.save("trained_ddpg_model.zip")